In [5]:
import os 
import json
from pycocotools.coco import COCO
from pycocotools import mask

from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon

from skimage import measure
from scipy import ndimage as ndi

from coco import visualise_anns, coco_to_binary, procAnns

ImportError: dlopen(/Users/soo-jeongkim/opt/anaconda3/envs/mphil_shape/lib/python3.7/site-packages/PIL/_imaging.cpython-37m-darwin.so, 2): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: /Users/soo-jeongkim/opt/anaconda3/envs/mphil_shape/lib/python3.7/site-packages/PIL/_imaging.cpython-37m-darwin.so
  Reason: Incompatible library version: _imaging.cpython-37m-darwin.so requires version 14.0.0 or later, but libjpeg.9.dylib provides version 12.0.0

### Processing annotations on h-minima imposition and filtering based on distance threshold

In [ ]:
annFile = "/content/drive/MyDrive/mphil/preprocessed_xfer_data/annotations/coco_first_annot-4.json"
img_dir = "drive/MyDrive/segmentation/xfer_images/proc_img_xfer/"

coco = COCO(annFile)
image_id = 59
img = coco.imgs[image_id]
img_ids = coco.getImgIds()
cat_ids = coco.getCatIds()
anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
anns = coco.loadAnns(anns_ids) # load anno

# FYI example on how to use -
img_dir = "drive/MyDrive/segmentation/xfer_images/proc_img_xfer/"
image = np.array(Image.open(os.path.join(img_dir, img['file_name'])))
plt.imshow(image, interpolation='nearest')
visualise_anns(anns)

# instead of - 
image = np.array(Image.open(os.path.join(img_dir, img['file_name'])))
plt.imshow(image, interpolation='nearest')
coco.showAnns(anns)

procAnns(anns)
anns_ids = [coco.getAnnIds(imgIds=id, catIds=cat_ids, iscrowd=None) for id in img_ids]
anns = coco.loadAnns(anns_ids) # load anno
# the new annotation file is called coco-annotations-5.json(something like this)

all_imganns = []
for id in img_ids:
  img = coco.imgs[id]
  anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
  anns = coco.loadAnns(anns_ids) # load anno
  ann_image = procAnns(anns)
  all_imganns.append(ann_image)

# Run 
coco = COCO(annFile)
img_ids = coco.getImgIds()
cat_ids = coco.getCatIds()

all_imganns = []
for id in img_ids:
  img = coco.imgs[id]
  anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
  anns = coco.loadAnns(anns_ids) # load anno
  annotations = procAnns(anns)
  all_imganns.append(annotations)

flat_list = [x for xs in all_imganns for x in xs]

with open(annFile, 'r') as f:
  ann_dict = json.load(f)

ann_dict['annotations'] = flat_list
# save new dict
with open("/content/drive/MyDrive/LIVECell/annotations/coco_first_annot-11.json", "w") as json_file:
    json.dump(ann_dict, json_file)

# Sanity check
new_ann =  "/content/drive/MyDrive/LIVECell/annotations/coco_first_annot-11.json"

coco = COCO(new_ann)
img_ids = coco.getImgIds()
image_id = 59
img = coco.imgs[image_id]
cat_ids = coco.getCatIds()
anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
anns = coco.loadAnns(anns_ids) # load anno
# annotations = procAnns(anns) # this is annotation of the last mask in the image

image = np.array(Image.open(os.path.join(img_dir, img['file_name'])))
plt.imshow(image, interpolation='nearest')
coco.showAnns(anns)